# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import json

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
# conn.set_session(autocommit=True)

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [5]:
filepath = song_files[0]
print(filepath)

/home/workspace/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json


In [12]:
# df = pd.read_json(filepath, typ='Series')
df = pd.read_json(filepath, lines=True)
# df.head()
df[['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()

[['SOMJBYD12A6D4F8557',
  'Keepin It Real (Skit)',
  'ARD0S291187B9B7BF5',
  0,
  114.78159]]

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [14]:
# song_data = list(df[['song_id', 'title', 'artist_id', 'year', 'duration']].values)
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values
song_data=song_data[0]
song_data=song_data.tolist()
print(song_data)

['SOMJBYD12A6D4F8557', 'Keepin It Real (Skit)', 'ARD0S291187B9B7BF5', 0, 114.78159]


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [16]:


# try: 
#     cur.execute(song_table_insert, song_data)
# except psycopg2.Error as e: 
#     print("Error: Inserting Rows")
#     print (e)
cur.execute(song_table_insert, song_data)
conn.commit()
cur.execute('SELECT * FROM songs')
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()


IntegrityError: duplicate key value violates unique constraint "songs_pkey"
DETAIL:  Key (song_id)=(SOMJBYD12A6D4F8557) already exists.


In [17]:
cur.execute("ROLLBACK")

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values
artist_data = artist_data.tolist()[0]
artist_data

['ARD0S291187B9B7BF5', 'Rated R', 'Ohio', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()
cur.execute("SELECT * FROM artists")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

('ARD0S291187B9B7BF5', 'Rated R', 'Ohio', nan, nan)


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files("data/log_data")

In [12]:
filepath = log_files[0]
print(filepath)

/home/workspace/data/log_data/2018/11/2018-11-29-events.json


In [13]:
df = pd.read_json(filepath, lines=True)
df.tail()
# df['auth'].unique()
# df['level'].unique()
# df['page'].unique()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
361,None,Logged In,Tegan,F,37,Levine,NaN,paid,"Portland-South Portland, ME",GET,Home,1.540794e+12,1065,None,200,1543532195796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
362,Pearl Jam,Logged In,Tegan,F,38,Levine,283.89832,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Rearviewmirror,200,1543532242796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
363,The 2 Live Crew,Logged In,Tegan,F,39,Levine,188.31628,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Me So Horny,200,1543532525796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
364,311,Logged In,Tegan,F,40,Levine,113.26649,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Loco,200,1543532713796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
365,Light Of The World,Logged In,Kynnedi,F,0,Sanchez,207.07220,free,"Cedar Rapids, IA",PUT,NextSong,1.541079e+12,1029,"Midnight Groovin' (7"" Version) (2006 Digital R...",200,1543533515796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5...",89


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df[df['page']=='NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [15]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-29 00:00:57.796
1   2018-11-29 00:01:30.796
2   2018-11-29 00:04:01.796
3   2018-11-29 00:04:55.796
4   2018-11-29 00:07:13.796
Name: ts, dtype: datetime64[ns]

In [16]:
time_data = pd.concat([t, t.dt.hour,t.dt.day,t.dt.week,t.dt.month, t.dt.year,t.dt.weekday], axis=1)
# print(time_data)
column_labels = ['timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [17]:
time_df = pd.DataFrame(data=time_data.values, columns=column_labels)
time_df.tail()

,timestamp,hour,day,week,month,year,weekday
314,2018-11-29 22:53:32.796000,22,29,48,11,2018,3
315,2018-11-29 22:57:22.796000,22,29,48,11,2018,3
316,2018-11-29 23:02:05.796000,23,29,48,11,2018,3
317,2018-11-29 23:05:13.796000,23,29,48,11,2018,3
318,2018-11-29 23:18:35.796000,23,29,48,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,73,Jacob,Klein,M,paid
1,24,Layla,Griffin,F,paid
2,24,Layla,Griffin,F,paid
3,73,Jacob,Klein,M,paid
4,24,Layla,Griffin,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [24]:
cur.execute("ROLLBACK")

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [21]:
from sql_queries import *
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (pd.to_datetime(row.ts, unit='ms'),int(row.userId),row.level, 
                               songid, artistid, row.sessionId,row.location, row.userAgent)

    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [22]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.